<a href="https://colab.research.google.com/github/Allen001822480/Info7374SpringTeam5/blob/Assignment4/Assignment4_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
tw = open('./TwitterLowerAsciiCorpus.txt')
twitter = tw.read()
data = [d for d in twitter.split('\n')]
data = [d for d in data if d != '']
#data = eval('[%s]'%repr(data).replace('[', '').replace(']', ''))
data = list(map(lambda x:re.sub(r'^A-Za-z\d\s\,\.\!\?\'\"\+\-','',x), data))
print(data[0:5])

["what's up dadyo when did you get back on twitter? haha", "like 2 weeks ago and it's going as terribly as i remember, but deg is still hilarious so it's ok", 'literally never about that account, love it.', 'answer me this fellow apple peoples: how many times in the past year have you used the escape key?', 'about 50 times today. terminal vim user.']


In [0]:
l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'STA', 'END', 'sta', 'end']
l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '']

for i, raw_word in enumerate(data):
    for j, term in enumerate(l1):
        raw_word = raw_word.replace(term,l2[j])
    
    data[i] = raw_word.lower()
    

In [0]:
data = list(map(lambda x:'STA '+x+' END', data))
context = data[::2]
answers = data[1::2]
all = context + answers

In [0]:
import nltk
import itertools
from keras.preprocessing import sequence
vocabulary_file = 'vocabulary_twitter'
padded_context_file = 'Padded_context'
padded_answers_file = 'Padded_answers'
unknown_token = 'something'

vocabulary_size = 10000
max_features = vocabulary_size
maxlen_input = 50
maxlen_output = 50  # cut texts after this number of words

all = ' '.join(all)
tokenized_all = all.split()
tokenized_context = [t.split() for t in context]
tokenized_answers = [t.split() for t in answers]

word_freq = nltk.FreqDist(itertools.chain(tokenized_all))
print ("Found %d unique words tokens." % len(word_freq.items()))


Using TensorFlow backend.


Found 11854 unique words tokens.


In [0]:
import pickle
vocab = word_freq.most_common(vocabulary_size-1)
with open(vocabulary_file, 'wb') as v:
  pickle.dump(vocab, v)

print(vocab[0:20])

[('STA', 10514), ('END', 10514), ('.', 6665), ('i', 5236), ('the', 2913), ('to', 2577), ('you', 2550), ('is', 2536), ('!', 2242), (',', 2213), ('it', 2051), ('a', 2046), ('not', 1964), ('?', 1839), ('and', 1655), ('that', 1436), ('my', 1196), ('in', 1171), ('of', 1082), ('am', 1056)]


In [0]:
vocab = pickle.load(open(vocabulary_file, 'rb'))
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print ("Using vocabulary of size %d." % vocabulary_size)
print ("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))


Using vocabulary of size 10000.
The least frequent word in our vocabulary is 'mcdonalds' and appeared 1 times.


In [0]:
import numpy as np
# Replacing all words not in our vocabulary with the unknown token:
for i, sent in enumerate(tokenized_answers):
  tokenized_answers[i] = [w if w in word_to_index else unknown_token for w in sent]
   
for i, sent in enumerate(tokenized_context):
  tokenized_context[i] = [w if w in word_to_index else unknown_token for w in sent]

# Creating the training data:
X = np.asarray([[word_to_index[w] for w in sent] for sent in tokenized_context])
Y = np.asarray([[word_to_index[w] for w in sent] for sent in tokenized_answers])

Q = sequence.pad_sequences(X, maxlen=maxlen_input, padding='post')
A = sequence.pad_sequences(Y, maxlen=maxlen_output, padding='post')

print(Q.shape)

with open(padded_context_file, 'wb') as q:
    pickle.dump(Q, q)
    
with open(padded_answers_file, 'wb') as a:
    pickle.dump(A, a)

(5257, 50)


In [0]:
import requests
file_url = "http://nlp.stanford.edu/data/glove.6B.zip"
tr = requests.get(file_url, stream=True)
with open("glove.6B.zip", "wb") as f:
    for chunk in tr.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)
            
            
import zipfile
import os
def un_zip(file_name):
    """unzip zip file"""
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(file_name + "_files"):
        pass
    else:
        os.mkdir(file_name + "_files")
    for names in zip_file.namelist():
        zip_file.extract(names,file_name + "_files/")
    zip_file.close()
        
glove = un_zip("glove.6B.zip")

In [0]:
padded_context_file = 'Padded_context'
padded_answers_file = 'Padded_answers'
unknown_token = 'something'
word_embedding_size = 100
sentence_embedding_size = 300
dictionary_size = 10000
maxlen_input = 50
maxlen_output = 50
num_subsets = 2
Epochs = 10
BatchSize = 128 
Patience = 0
dropout = .25
n_test = 100

In [0]:
import _pickle
import numpy as np

embeddings_index = {}
f = open('./glove.6B.zip_files/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((dictionary_size, word_embedding_size))

# Loading our vocabulary:
vocabulary = _pickle.load(open(vocabulary_file, 'rb'))

# Using the Glove embedding:
i = 0
for word in vocabulary:
    embedding_vector = embeddings_index.get(word[0])
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    i += 1
    
print(embedding_matrix)

Found 400000 word vectors.
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.33978999  0.20941     0.46348    ... -0.23394001  0.47297999
  -0.028803  ]
 ...
 [-0.16700999  0.10193    -0.62102997 ... -0.55523002 -0.60065001
  -0.22685   ]
 [ 0.31239     0.031386   -0.27726999 ...  0.28920999  0.82100999
   0.84512001]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [0]:
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout, concatenate
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
import keras.backend as K
import os
import theano.tensor as T
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt

weights_file = 'assignment4_twitter.h5'
ad = Adam(lr=0.00005) 

input_context = Input(shape=(maxlen_input,), name='input_context')
input_answer = Input(shape=(maxlen_input,), name='input_answer')
LSTM_encoder = LSTM(sentence_embedding_size, init= 'lecun_uniform')
LSTM_decoder = LSTM(sentence_embedding_size, init= 'lecun_uniform')
if os.path.isfile(weights_file):
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, input_length=maxlen_input)
else:
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, weights=[embedding_matrix], input_length=maxlen_input)
word_embedding_context = Shared_Embedding(input_context)
context_embedding = LSTM_encoder(word_embedding_context)

word_embedding_answer = Shared_Embedding(input_answer)
answer_embedding = LSTM_decoder(word_embedding_answer)

merge_layer = concatenate([context_embedding, answer_embedding])
out = Dense(int(dictionary_size/2), activation="relu")(merge_layer)
out = Dense(dictionary_size, activation="softmax")(out)

model = Model(input=[input_context, input_answer], output = [out])

model.compile(loss='categorical_crossentropy', optimizer=ad)

if os.path.isfile(weights_file):
    model.load_weights(weights_file)
    
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="lecun_uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="lecun_uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_context (InputLayer)      (None, 50)           0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 50, 100)      1000000     input_context[0][0]              
                                                                 input_answer[0][0]               
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 300)          481200      embedding_2[0][0]                
__________

In [0]:
q = _pickle.load(open(padded_context_file, 'rb'))
a = _pickle.load(open(padded_answers_file, 'rb'))
n_exem, n_words = a.shape

qt = q[0:n_test,:]
at = a[0:n_test,:]
q = q[n_test + 1:,:]
a = a[n_test + 1:,:]

print('Number of exemples = %d'%(n_exem - n_test))
step = int(np.around((n_exem - n_test)/num_subsets))
round_exem = int(step * num_subsets)
print(step, round_exem)

Number of exemples = 5157
2578 5156


In [0]:
def print_result(input):
  ans_partial = np.zeros((1,maxlen_input))
  ans_partial[0,-1] = 0 #index of STA
  for k in range(maxlen_input - 1):
    ye = model.predict([input, ans_partial])
    mp = np.argmax(ye)
    ans_partial[0, 0:-1] = ans_partial[0, 1:]
    ans_partial[0, -1] = mp
  text = ''
  for k in ans_partial[0]:
    k = k.astype(int)
    if k < (dictionary_size-2):
      w = vocabulary[k]
      text = text + w[0] + ' '
  return(text)
  

In [0]:
x = range(0,Epochs) 
valid_loss = np.zeros(Epochs)
train_loss = np.zeros(Epochs)
for m in range(Epochs):
  # Loop over training batches due to memory constraints:
  for n in range(0,round_exem,step):
    q2 = q[n:n+step]
    s = q2.shape
    count = 0
    for i, sent in enumerate(a[n:n+step]):
      l = np.where(sent==1)
      limit = l[0][0]
      count += limit + 1
    Q = np.zeros((count,maxlen_input))
    A = np.zeros((count,maxlen_input))
    Y = np.zeros((count,dictionary_size))
    
    count = 0
    for i, sent in enumerate(a[n:n+step]):
      ans_partial = np.zeros((1,maxlen_input))
      # Loop over the positions of the current target output (the current output sequence):
      l = np.where(sent==1)
      limit = l[0][0]
      for k in range(1,limit+1):
        # Mapping the target output (the next output word) for one-hot codding:
        y = np.zeros((1, dictionary_size))
        y[0, sent[k]] = 1
        # preparing the partial answer to input:
        ans_partial[0,-k:] = sent[0:k]
        # training the model for one epoch using teacher forcing:
        Q[count, :] = q2[i:i+1] 
        A[count, :] = ans_partial 
        Y[count, :] = y
        count += 1
    print('Training epoch: %d, training examples: %d - %d'%(m,n, n + step))
    
    model.fit([Q, A], Y, batch_size=BatchSize, epochs=1)
    test_input = qt[6:7]
    print(print_result(test_input))
  model.save_weights(weights_file, overwrite=True)

Training epoch: 0, training examples: 0 - 2578
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
36329/36329 [==============================] - 89s 2ms/step - loss: 6.3038
STA END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END 
Training epoch: 0, training examples: 2578 - 5156
Epoch 1/1
37537/37537 [==============================] - 89s 2ms/step - loss: 5.5544
STA i i am am END END END END i am END END END i am END END END i am END END END i am END END END i am END END END i am END END END i am END END END i am END END END i 
Training epoch: 1, training examples: 0 - 2578
Epoch 1/1
36329/36329 [==============================] - 86s 2ms/step - loss: 5.4789
STA i am not not not not not not not not not not not not to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to 
Training epoch: 1, t

In [0]:
for i in range(50):
  print('input: %s'%context[i].replace('STA','').replace('END',''))
  output = print_result(qt[i:i+1])
  print('output: %s'%output)
  i += 1
  

input:  what is up dadyo when did you get back on twitter ? haha 
output: STA i am so happy for you END END END END END END END END END i am END END END END END END i have not played in the middle of the . END END END END END END END END END END END END END END 
input:  literally never about that account , love it . 
output: STA i am so happy for the first episode of the END END END END END END END END END END END END END END END END 
input:  about 50 times today . terminal vim user . 
output: STA i am not sure . . . . . . . END END END END . END . END . END . END END END END END END END END END END END END END END END END END END END END END END 
input:  cmd+opt+esc is good but still available via menubar 
output: STA i am not sure . . . . . . . END END END END . END . END . END END END END END END END END END END END END END END END END END END END END END END 
input:  i am disgusted 
output: STA i am not a person . . . . . . END END , so , i will not have a fan . END END END END END

In [0]:
x = range(0,Epochs) 
valid_loss = np.zeros(Epochs)
train_loss = np.zeros(Epochs)
for m in range(Epochs):
  # Loop over training batches due to memory constraints:
  for n in range(0,round_exem,step):
    q2 = q[n:n+step]
    s = q2.shape
    count = 0
    for i, sent in enumerate(a[n:n+step]):
      l = np.where(sent==1)
      limit = l[0][0]
      count += limit + 1
    Q = np.zeros((count,maxlen_input))
    A = np.zeros((count,maxlen_input))
    Y = np.zeros((count,dictionary_size))
    
    count = 0
    for i, sent in enumerate(a[n:n+step]):
      ans_partial = np.zeros((1,maxlen_input))
      # Loop over the positions of the current target output (the current output sequence):
      l = np.where(sent==1)
      limit = l[0][0]
      for k in range(1,limit+1):
        # Mapping the target output (the next output word) for one-hot codding:
        y = np.zeros((1, dictionary_size))
        y[0, sent[k]] = 1
        # preparing the partial answer to input:
        ans_partial[0,-k:] = sent[0:k]
        # training the model for one epoch using teacher forcing:
        Q[count, :] = q2[i:i+1] 
        A[count, :] = ans_partial 
        Y[count, :] = y
        count += 1
    print('Training epoch: %d, training examples: %d - %d'%(m,n, n + step))
    model.fit([Q, A], Y, batch_size=BatchSize, epochs=1)
    test_input = qt[6:7]
    print(print_result(test_input))
  model.save_weights(weights_file, overwrite=True)

Training epoch: 0, training examples: 0 - 2578
Epoch 1/1
36329/36329 [==============================] - 88s 2ms/step - loss: 2.4340
STA i am sayinggg jealous END END END END END i have been played in the middle of the night END END END END END END END END END END END END END END END END END END END END END END END END END END END END i know 
Training epoch: 0, training examples: 2578 - 5156
Epoch 1/1
37537/37537 [==============================] - 89s 2ms/step - loss: 2.4508
STA i am not feelin class END END END END i have not played the in the league END END END END END END END END END END END END END END END END END END 
Training epoch: 1, training examples: 0 - 2578
Epoch 1/1
36329/36329 [==============================] - 86s 2ms/step - loss: 2.3316
STA i have ben , i had be at the day . i am not talking about it is a big day . END . END END END END END END END END END END END END END END END END END END END END END END END END END 
Training epoch: 1, training examples: 2578 - 5156
E

In [0]:
for i in range(50):
  print('input: %s'%context[i].replace('STA','').replace('END',''))
  output = print_result(qt[i:i+1])
  print('output: %s'%output)
  i += 1
  

input:  what is up dadyo when did you get back on twitter ? haha 
output: STA i am not a spurs fan . END him &amp ; kawhi are so so well i have to do END END END END END END END END END END END END END END END END END END END END END END END END END END END END END 
input:  literally never about that account , love it . 
output: STA i am not sure what is working out well . i am trying to do real shit out at the game . END END END END END END END END END END END END END END END END END END END END END END END END END END END 
input:  about 50 times today . terminal vim user . 
output: STA i am not sure what is working out of these keep those days . END is END END END END END END END END END END END END END END END END END END END END END END END END END END END END END ? END 
input:  cmd+opt+esc is good but still available via menubar 
output: STA i am not sure what is working out of those these days lol END i am ready to rt and go these day END END END END END END END END END END END E

In [0]:
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout, concatenate
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
import keras.backend as K
import os
import theano.tensor as T
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt

weights_file = 'assignment4_twitter.h5'
ad = Adam(lr=0.00005) 

input_context = Input(shape=(maxlen_input,), name='input_context')
input_answer = Input(shape=(maxlen_input,), name='input_answer')
LSTM_encoder = LSTM(sentence_embedding_size, init= 'lecun_uniform')
LSTM_decoder = LSTM(sentence_embedding_size, init= 'lecun_uniform')
if os.path.isfile(weights_file):
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, input_length=maxlen_input)
else:
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, weights=[embedding_matrix], input_length=maxlen_input)
word_embedding_context = Shared_Embedding(input_context)
context_embedding = LSTM_encoder(word_embedding_context)

word_embedding_answer = Shared_Embedding(input_answer)
answer_embedding = LSTM_decoder(word_embedding_answer)

merge_layer = concatenate([context_embedding, answer_embedding])
out = Dense(int(dictionary_size/2), activation="relu")(merge_layer)
out = Dense(dictionary_size, activation="softmax")(out)

model = Model(input=[input_context, input_answer], output = [out])

model.compile(loss='categorical_crossentropy', optimizer=ad)

if os.path.isfile(weights_file):
    model.load_weights(weights_file)
    
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="lecun_uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="lecun_uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_context (InputLayer)      (None, 50)           0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 50, 100)      1000000     input_context[0][0]              
                                                                 input_answer[0][0]               
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 300)          481200      embedding_3[0][0]                
__________

In [0]:
x = range(0,Epochs) 
valid_loss = np.zeros(Epochs)
train_loss = np.zeros(Epochs)
for m in range(Epochs):
  # Loop over training batches due to memory constraints:
  for n in range(0,round_exem,step):
    q2 = q[n:n+step]
    s = q2.shape
    count = 0
    for i, sent in enumerate(a[n:n+step]):
      l = np.where(sent==1)
      limit = l[0][0]
      count += limit + 1
    Q = np.zeros((count,maxlen_input))
    A = np.zeros((count,maxlen_input))
    Y = np.zeros((count,dictionary_size))
    
    count = 0
    for i, sent in enumerate(a[n:n+step]):
      ans_partial = np.zeros((1,maxlen_input))
      # Loop over the positions of the current target output (the current output sequence):
      l = np.where(sent==1)
      limit = l[0][0]
      for k in range(1,limit+1):
        # Mapping the target output (the next output word) for one-hot codding:
        y = np.zeros((1, dictionary_size))
        y[0, sent[k]] = 1
        # preparing the partial answer to input:
        ans_partial[0,-k:] = sent[0:k]
        # training the model for one epoch using teacher forcing:
        Q[count, :] = q2[i:i+1] 
        A[count, :] = ans_partial 
        Y[count, :] = y
        count += 1
    print('Training epoch: %d, training examples: %d - %d'%(m,n, n + step))
    model.fit([Q, A], Y, batch_size=BatchSize, epochs=1)
    test_input = qt[6:7]
    print(print_result(test_input))
  model.save_weights(weights_file, overwrite=True)

Training epoch: 0, training examples: 0 - 2578
Epoch 1/1
36329/36329 [==============================] - 89s 2ms/step - loss: 1.2059
STA i do not know what the mouse was thinking ? ? why come into a house with 2 ferocious felines ? ? END END END END END END END END END END END END END END END END END END END END END END END END END END END 
Training epoch: 0, training examples: 2578 - 5156
Epoch 1/1
37537/37537 [==============================] - 89s 2ms/step - loss: 1.2007
STA i am not feelin class tomorrow rn but i definitely do not have a huge deal with my collection END END END END END END END END END END END END END END END END END END END END END END END END END END END END END . 
Training epoch: 1, training examples: 0 - 2578
Epoch 1/1
36329/36329 [==============================] - 86s 2ms/step - loss: 1.1445
STA i do not know what the mouse was thinking ? ? why come into a house with 2 ferocious felines ? ? END END END END END END END END END END END END END END END END END END E

In [0]:
for i in range(20):
  print('input: %s'%context[i].replace('STA','').replace('END',''))
  output = print_result(q[i:i+1])
  print('output: %s'%output)
  i += 1
  

input:  what is up dadyo when did you get back on twitter ? haha 
output: STA creepy . END END END . do you are at the time ? END ? END are you END END END END END END END END END END END END END END END END END END END END . END END END . END END END END END . 
input:  literally never about that account , love it . 
output: STA i followed the END and i am not even this of the . END is not mean END END END END END END END END END END END END END END END END END END END END END i know END is the 
input:  about 50 times today . terminal vim user . 
output: STA i am sorry i will continue my love for the old show END END END END END END END END END END END END END END . END . END . END END END END END END . END END END END END END END END END END END 
input:  cmd+opt+esc is good but still available via menubar 
output: STA switch or hand jane . . . .choose ! ! ! ! END up those x ! END END END END END ; now ! ! ! END ! END ;3 END END END END END END END END ! END END END END END END ! ! 
in

In [0]:
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout, concatenate
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
import keras.backend as K
import os
import theano.tensor as T
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt

weights_file = 'assignment4_twitter.h5'
ad = Adam(lr=0.00005) 

input_context = Input(shape=(maxlen_input,), name='input_context')
input_answer = Input(shape=(maxlen_input,), name='input_answer')
LSTM_encoder = LSTM(sentence_embedding_size, init= 'lecun_uniform')
LSTM_decoder = LSTM(sentence_embedding_size, init= 'lecun_uniform')
if os.path.isfile(weights_file):
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, input_length=maxlen_input)
else:
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, weights=[embedding_matrix], input_length=maxlen_input)
word_embedding_context = Shared_Embedding(input_context)
context_embedding = LSTM_encoder(word_embedding_context)

word_embedding_answer = Shared_Embedding(input_answer)
answer_embedding = LSTM_decoder(word_embedding_answer)

merge_layer = concatenate([context_embedding, answer_embedding])
out = Dense(int(dictionary_size/2), activation="relu")(merge_layer)
out = Dense(dictionary_size, activation="softmax")(out)

model = Model(input=[input_context, input_answer], output = [out])

model.compile(loss='categorical_crossentropy', optimizer=ad)

if os.path.isfile(weights_file):
    model.load_weights(weights_file)
    
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="lecun_uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="lecun_uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_context (InputLayer)      (None, 50)           0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 50, 100)      1000000     input_context[0][0]              
                                                                 input_answer[0][0]               
__________________________________________________________________________________________________
lstm_9 (LSTM)                   (None, 300)          481200      embedding_5[0][0]                
__________

In [0]:
x = range(0,Epochs) 
valid_loss = np.zeros(Epochs)
train_loss = np.zeros(Epochs)
for m in range(Epochs):
  # Loop over training batches due to memory constraints:
  for n in range(0,round_exem,step):
    q2 = q[n:n+step]
    s = q2.shape
    count = 0
    for i, sent in enumerate(a[n:n+step]):
      l = np.where(sent==1)
      limit = l[0][0]
      count += limit + 1
    Q = np.zeros((count,maxlen_input))
    A = np.zeros((count,maxlen_input))
    Y = np.zeros((count,dictionary_size))
    
    count = 0
    for i, sent in enumerate(a[n:n+step]):
      ans_partial = np.zeros((1,maxlen_input))
      # Loop over the positions of the current target output (the current output sequence):
      l = np.where(sent==1)
      limit = l[0][0]
      for k in range(1,limit+1):
        # Mapping the target output (the next output word) for one-hot codding:
        y = np.zeros((1, dictionary_size))
        y[0, sent[k]] = 1
        # preparing the partial answer to input:
        ans_partial[0,-k:] = sent[0:k]
        # training the model for one epoch using teacher forcing:
        Q[count, :] = q2[i:i+1] 
        A[count, :] = ans_partial 
        Y[count, :] = y
        count += 1
    print('Training epoch: %d, training examples: %d - %d'%(m,n, n + step))
    model.fit([Q, A], Y, batch_size=BatchSize, epochs=1)
    test_input = qt[6:7]
    print(print_result(test_input))
  model.save_weights(weights_file, overwrite=True)

Training epoch: 0, training examples: 0 - 2578
Epoch 1/1
36329/36329 [==============================] - 90s 2ms/step - loss: 0.5484
STA i do not know what the mouse was thinking ? ? why come into a house with 2 ferocious felines ? ? END END END END END END END END END END END END END END END END END END END END END the flight is the new day 
Training epoch: 0, training examples: 2578 - 5156
Epoch 1/1
37537/37537 [==============================] - 89s 2ms/step - loss: 0.5244
STA good morning gorgeous how are u today babe ? END . END END END END END END END , i got it END END END END END END END END END END END END END END END END . END END END . END END END END END . 
Training epoch: 1, training examples: 0 - 2578
Epoch 1/1
36329/36329 [==============================] - 87s 2ms/step - loss: 0.5166
STA i do not know what the mouse was thinking ? ? why come into a house with 2 ferocious felines ? ? END END END END END END END END END END END END END END END END END END END END END the fac

In [0]:
for i in range(20):
  print('input: %s'%context[i].replace('STA','').replace('END',''))
  output = print_result(q[i:i+1])
  print('output: %s'%output)
  i += 1

input:  what is up dadyo when did you get back on twitter ? haha 
output: STA END END END can not forget END END END END . . one that may as the person can you do END END END END END END END END END END END END END END END END END END END END END END . END END END 
input:  literally never about that account , love it . 
output: STA i followed the END END i have read it but its but i am gonna take it off it END END END END END . END . END END END END END END END END END END END END END END END END END END . END 
input:  about 50 times today . terminal vim user . 
output: STA i am sorry i will not continue my respect for a cubbies and not fit . END . END it is a huge it END END END END END END END END END END END END END END END END END END END END END END END END 
input:  cmd+opt+esc is good but still available via menubar 
output: STA switch or hand jane . . . .choose ! ! ! ! END up x END END END ! ! END ! END ! END ;3 END END END END ; now END ? END END END END END END END END END END